In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import matplotlib.cm as cm
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
from prettytable import PrettyTable

# Load files
Here we load given data

In [ ]:
# %load grid.py
import bisect

## Grid parameters
# Top left corner of the area
MAP_LAT = 46.5
MAP_LON = 6.55

# Total area size
MAP_SIZE_LAT = 0.07
MAP_SIZE_LON = 0.10

# Number of cells
CELL_NUM_LAT = 10
CELL_NUM_LON = 10

# Grid lines
GRID_LAT_POINTS = [MAP_LAT + i * (MAP_SIZE_LAT / CELL_NUM_LAT)
                   for i in range(1, CELL_NUM_LAT + 1)]
GRID_LON_POINTS = [MAP_LON + i * (MAP_SIZE_LON / CELL_NUM_LON)
                   for i in range(1, CELL_NUM_LON + 1)]


def location_to_cell_id(lat, lon):
    """Get the grid cell ID for a given latitude and longitude."""
    if not (MAP_LAT <= lat < MAP_LAT + MAP_SIZE_LAT) or not (
        MAP_LON <= lon < MAP_LON + MAP_SIZE_LON
    ):
        raise ValueError("Out of area range.")

    i = bisect.bisect(GRID_LAT_POINTS, lat)
    j = bisect.bisect(GRID_LON_POINTS, lon)
    return i * CELL_NUM_LAT + j + 1


In [ ]:
f_queries = open("queries.csv", "r")
queries = f_queries.read()

f_pois = open("pois.csv", "r")
pois = f_pois.read()

# Functions to analyze data

In [ ]:
# some useful function

def is_weekend_day(float_nb):
    #return boolean of if the time is a time of a weekend day
    return (float_nb>=120 and float_nb<=168) or (float_nb>=288 and float_nb<=336) or (float_nb>=456)

def is_weekday(float_nb):
    #return boolean of if the time is a time of no-weekend day
    return not is_weekend_day(float_nb)

def is_worktime(float_nb):
    #worktime = between 9->12 and 14->17h 
    #return boolean: is the time during the slot of time where users are usually at work
    return is_weekday(float_nb)and(((float_nb%24>=9)and(float_nb%24<12))or((float_nb%24>=14)and(float_nb%24<17)))

def is_hometime(float_nb):
    #hometime = weekend or between 10PM -> 6AM
    #return boolean: is the time during the slot of time where users are usually at home
    return (is_weekday(float_nb)and((float_nb%24<=6) or (float_nb%24>=22)))
    #is_weekend_day(float_nb)or

In [ ]:
#some functions given a ip address: 
#here the format of everything_list is assumed to be list[list[ip_add, lat, lon, timestamp, poi]] everythin as string

def get_pois(ip, everything_list):
    #return set of pois of given IP adress (user)
    return set([x[4] for x in everything_list if(x[0]==ip)])

def get_positions(ip, everything_list):
    #return all position of an ip adress: the format is a tuple of (list(all_x_axe_positions), list(y_positions))
    l = [(float(x[1]), float(x[2])) for x in everything_list if(x[0]==ip)]
    return ([i for i,j in l], [j for i,j in l])

def get_positions_work(ip, everything_list):
    #return all position of an ip adress at work: 
    #the format is a tuple of (list(all_x_axe_positions), list(y_positions))
    l = [(float(x[1]), float(x[2])) for x in everything_list if(x[0]==ip and is_worktime(float(x[3])))]
    return ([i for i,j in l], [j for i,j in l])

def get_positions_home(ip, everything_list):
    #return all position of an ip adress at home: 
    #the format is a tuple of (list(all_x_axe_positions), list(y_positions))
    set_all = set([(float(x[1]), float(x[2])) for x in everything_list if(x[0]==ip)])
    set_work = set([(float(x[1]), float(x[2])) for x in everything_list if(x[0]==ip and is_worktime(float(x[3])))])
    #l = [(float(x[1]), float(x[2])) for x in everything_list if(x[0]==ip and not(is_worktime(float(x[3]))))]
    l =list(set_all-set_work)
    return ([i for i,j in l], [j for i,j in l])
    

def get_work_pois(ip, everything_list):
    #return POis a given IP adress look at when he is at work
    return set([x[4] for x in everything_list if(x[0]==ip and is_worktime(float(x[3])))])

def get_hobby_pois(ip, everything_list):
    #return POis a given IP adress look at when he is not at work
    return set([x[4] for x in everything_list if(x[0]==ip and not(is_worktime(float(x[3]))))])

def get_work_cell(ip, everything_list):
    #return cell id of a user when he is at work
    l = [(float(x[1]), float(x[2])) for x in everything_list if(x[0]==ip and is_worktime(float(x[3])))]
    return set([location_to_cell_id(x, y) for x,y in l])

def get_home_cell(ip, everything_list):
    #return cell id of a user when he is at work
    l = [(float(x[1]), float(x[2])) for x in everything_list if(x[0]==ip and is_hometime(float(x[3])))]
    return set([location_to_cell_id(x, y) for x,y in l])
    
    
    
    

In [ ]:
#loading all possible ip addresses
with open("queries.csv", "r") as f_queries:
    queries = f_queries.readlines()
    
    queries = [q.split() for q in queries][1:]
    
    ip_set = set([x[0] for x in queries])


In [ ]:
#loading all possible pois
with open("pois.csv", "r") as f_pois:
    pois = f_pois.readlines()
    pois = [p.split() for p in pois][1:]

all_pois = set([x[2] for x in pois])

print(len(all_pois))
print(all_pois)

# Graph drawing
first we create random colors for each user

In [ ]:
users_number = 10

In [ ]:
colors = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
             for i in range(users_number)]

## Total locations of users graph

In [ ]:
#total locations of n users 
data = [get_positions(ip, queries) for ip in list(ip_set)[:users_number]]


grid_elem_lat = [MAP_LAT]
grid_elem_lat.extend(GRID_LAT_POINTS)
lat_space = grid_elem_lat[1]-grid_elem_lat[0]


grid_elem_lon = [MAP_LON]
grid_elem_lon.extend(GRID_LON_POINTS)
lon_space = grid_elem_lon[3]-grid_elem_lon[2]


fig, ax = plt.subplots(figsize=(10, 8))


for data, color in zip(data, colors):
    x, y = data
    ax.scatter(x, y, c=color)


    
ax.set_xlim(grid_elem_lat[0], grid_elem_lat[10])
ax.set_ylim(grid_elem_lon[0], grid_elem_lon[10])

ax.set_xticks(grid_elem_lat)
ax.set_yticks(grid_elem_lon)



ax.set_xlabel("Latitude")
ax.set_ylabel("Longitude")


plt.grid(True)
#if you want to save the image
#plt.savefig('locations_home10.jpg')
plt.show()

## Home locations of users graph

In [ ]:
#home locations of the same n users 
data = [get_positions_home(ip, queries) for ip in list(ip_set)[:users_number]]


grid_elem_lat = [MAP_LAT]
grid_elem_lat.extend(GRID_LAT_POINTS)
lat_space = grid_elem_lat[1]-grid_elem_lat[0]


grid_elem_lon = [MAP_LON]
grid_elem_lon.extend(GRID_LON_POINTS)
lon_space = grid_elem_lon[3]-grid_elem_lon[2]


fig, ax = plt.subplots(figsize=(10, 8))


for data, color in zip(data, colors):
    x, y = data
    ax.scatter(x, y, c=color)


    
ax.set_xlim(grid_elem_lat[0], grid_elem_lat[10])
ax.set_ylim(grid_elem_lon[0], grid_elem_lon[10])

ax.set_xticks(grid_elem_lat)
ax.set_yticks(grid_elem_lon)



ax.set_xlabel("Latitude")
ax.set_ylabel("Longitude")


plt.grid(True)
#if you want to save the image
#plt.savefig('locations_home10.jpg')
plt.show()

## Work locations of users graph

In [ ]:
#work locations of the same n users 
data = [get_positions_work(ip, queries) for ip in list(ip_set)[:users_number]]


grid_elem_lat = [MAP_LAT]
grid_elem_lat.extend(GRID_LAT_POINTS)
lat_space = grid_elem_lat[1]-grid_elem_lat[0]


grid_elem_lon = [MAP_LON]
grid_elem_lon.extend(GRID_LON_POINTS)
lon_space = grid_elem_lon[3]-grid_elem_lon[2]


fig, ax = plt.subplots(figsize=(10, 8))


for data, color in zip(data, colors):
    x, y = data
    ax.scatter(x, y, c=color)


    
ax.set_xlim(grid_elem_lat[0], grid_elem_lat[10])
ax.set_ylim(grid_elem_lon[0], grid_elem_lon[10])

ax.set_xticks(grid_elem_lat)
ax.set_yticks(grid_elem_lon)



ax.set_xlabel("Latitude")
ax.set_ylabel("Longitude")


plt.grid(True)
#if you want to save the image
#plt.savefig('locations_home10.jpg')
plt.show()

# POis table of users

In [ ]:
users_number = 10

data = [[ip, get_pois(ip, queries)] for ip in list(ip_set)[:users_number]]

t = PrettyTable(['ip address', 'POIs'])

for row in data:
    t.add_row(row)
    
print(t)